In [1]:
import pandas as pd
import numpy as np

In [2]:
## !pip install pyarrow

In [2]:
import pickle

In [3]:
import mlflow

In [4]:
mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('nyc-experiment')

2024/06/28 08:06:31 INFO mlflow.tracking.fluent: Experiment with name 'nyc-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/workspaces/MLOps-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1719561991637, experiment_id='1', last_update_time=1719561991637, lifecycle_stage='active', name='nyc-experiment', tags={}>

In [5]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [6]:
import seaborn as sns

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

In [7]:
df.shape

(3066766, 19)

In [8]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [9]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [10]:
len(df_train), len(df_val)

(3009173, 2855951)

# Reduced the size of train and test data due to buffer size issue and hence the rmse value may vary

In [11]:
df_train = df_train.iloc[:20000] 
df_val = df_val.iloc[:20000] 

In [12]:
len(df_train), len(df_val)

(20000, 20000)

In [13]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [14]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

In [15]:
dv = DictVectorizer()

In [16]:
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

In [17]:
X_train.shape

(20000, 4506)

In [18]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [19]:
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

Training the model

In [20]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

Evaluating the model

In [21]:
y_pred = lr.predict(X_val)

rmse_val = mean_squared_error(y_val, y_pred, squared=False)
rmse_val

/home/codespace/anaconda3/envs/exp-track-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


6.1952743368852845

In [22]:
from sklearn.linear_model import Lasso

In [23]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [25]:
lr = Lasso(0.1)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)
rmse = mean_squared_error(y_val, y_pred, squared=False)

/home/codespace/anaconda3/envs/exp-track-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [26]:
rmse

5.737033794695259

In [28]:
with mlflow.start_run():

    mlflow.set_tag("developer", "Bhavana")

    #mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    #mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

/home/codespace/anaconda3/envs/exp-track-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [29]:
import xgboost as xgb

In [30]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [31]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [32]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [33]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

  0%|                                                              | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/codespace/anaconda3/envs/exp-track-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [08:36:39] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.56356                                                                               
[1]	validation-rmse:6.46786                                                                               
[2]	validation-rmse:5.94130                                                                               
[3]	validation-rmse:5.69510                                                                               
[4]	validation-rmse:5.59209                                                                               
[5]	validation-rmse:5.54693                                                                               
[6]	validation-rmse:5.53263                                                                               
[7]	validation-rmse:5.52593                                                                               
[8]	validation-rmse:5.53347                                                                               
[9]	validation-rmse:5.54653          

/home/codespace/anaconda3/envs/exp-track-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/home/codespace/anaconda3/envs/exp-track-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [08:36:45] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.00736                                                                               
[1]	validation-rmse:6.97799                                                                               
[2]	validation-rmse:6.36243                                                                               
[3]	validation-rmse:6.01288                                                                               
[4]	validation-rmse:5.81695                                                                               
[5]	validation-rmse:5.71555                                                                               
[6]	validation-rmse:5.66286                                                                               
[7]	validation-rmse:5.64510                                                                               
[8]	validation-rmse:5.64315                                                                               
[9]	validation-rmse:5.65418          

...

/home/codespace/anaconda3/envs/exp-track-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



[0]	validation-rmse:8.64962                                                                               
[1]	validation-rmse:7.86343                                                                               
[2]	validation-rmse:7.25496                                                                               
 98%|███████████████████████████████████▎| 49/50 [06:02<00:05,  5.12s/trial, best loss: 5.412425829765289]

/home/codespace/anaconda3/envs/exp-track-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [08:42:41] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[3]	validation-rmse:6.78991                                                                               
[4]	validation-rmse:6.43866                                                                               
[5]	validation-rmse:6.17614                                                                               
[6]	validation-rmse:5.98044                                                                               
[7]	validation-rmse:5.83713                                                                               
[8]	validation-rmse:5.73086                                                                               
[9]	validation-rmse:5.65265                                                                               
[10]	validation-rmse:5.59507                                                                              
[11]	validation-rmse:5.55263                                                                              
[12]	validation-rmse:5.52097         

/home/codespace/anaconda3/envs/exp-track-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



Get the best rmse params from mlflow ui and replace the params values in search space and run the mlflow run

In [39]:
with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        best_params = {
            'learning_rate' : 0.7993188205339578,
            'max_depth' : 73,
            'min_child_weight' : 0.5694797340885676,
            'objective' : 'reg:linear',
            'reg_alpha' : 0.06481429405630112,
            'reg_lambda' : 0.20149788479279268,
            'seed' : 42
        }
        
        mlflow.log_params(best_params)
        booster = xgb.train(
            params=best_params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

        with open('models/preprocessor.b', 'wb') as f_out:
            pickle.dump(dv, f_out)
        mlflow.log_artifact('models/preprocessor.b', artifact_path='preprocessor')
    
        mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/home/codespace/anaconda3/envs/exp-track-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:15:09] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:6.16803
[1]	validation-rmse:6.06927
[2]	validation-rmse:6.08759
[3]	validation-rmse:6.08715
[4]	validation-rmse:6.09183
[5]	validation-rmse:6.12691
[6]	validation-rmse:6.12641
[7]	validation-rmse:6.13084
[8]	validation-rmse:6.13386
[9]	validation-rmse:6.13677
[10]	validation-rmse:6.13767
[11]	validation-rmse:6.13780
[12]	validation-rmse:6.13121
[13]	validation-rmse:6.13153
[14]	validation-rmse:6.13282
[15]	validation-rmse:6.13256
[16]	validation-rmse:6.13093
[17]	validation-rmse:6.13379
[18]	validation-rmse:6.13583
[19]	validation-rmse:6.13697
[20]	validation-rmse:6.14021
[21]	validation-rmse:6.14780
[22]	validation-rmse:6.15560
[23]	validation-rmse:6.15871
[24]	validation-rmse:6.16125
[25]	validation-rmse:6.16244
[26]	validation-rmse:6.16508
[27]	validation-rmse:6.16788
[28]	validation-rmse:6.17605
[29]	validation-rmse:6.17858
[30]	validation-rmse:6.18407
[31]	validation-rmse:6.18994
[32]	validation-rmse:6.19217
[33]	validation-rmse:6.19261
[34]	validation-rmse:6.1

/home/codespace/anaconda3/envs/exp-track-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/codespace/anaconda3/envs/exp-track-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:15:16] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


In [42]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR

#mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, LinearSVR):

    with mlflow.start_run():

        with open('models/preprocessor.b', 'wb') as f_out:
            pickle.dump(dv, f_out)
        mlflow.log_artifact('models/preprocessor.b', artifact_path='preprocessor')

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

/home/codespace/anaconda3/envs/exp-track-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/codespace/anaconda3/envs/exp-track-env/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
